<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "hN2itDTj1upyev1f.yLm5qtPpASQXEv7GJA79T2c6wSUnsNCNFxgRMbmnSbJHA1i31wgrSgmnAEVmb4wo"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169556
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "stockholm", "street": "stockholm-st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality2',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [4]:
import datetime
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
data_df = air_quality_fg.read()

current_date = aq_today_df['date'].iloc[0]  
current_date = pd.to_datetime(current_date)  
yesterday = current_date - pd.Timedelta(days=1)
yesterday_formatted = yesterday.isoformat() + "+00:00"  


yesterday_row = data_df.loc[data_df['date'] == yesterday_formatted]


if not yesterday_row.empty:
    pm25_lag_1 = yesterday_row['pm25'].values[0]
    pm25_lag_2 = yesterday_row['pm25_lag_1'].values[0]
    pm25_lag_3 = yesterday_row['pm25_lag_2'].values[0]
    
    aq_today_df['pm25_lag_1'] = pm25_lag_1
    aq_today_df['pm25_lag_2'] = pm25_lag_2
    aq_today_df['pm25_lag_3'] = pm25_lag_3
else:
    print("No data found for yesterday's date in data_df.")

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.18s) 


,pm25,country,city,street,date,url,pm25_lag_1,pm25_lag_2,pm25_lag_3
0,28.0,sweden,stockholm,stockholm-st-eriksgatan-83,2024-11-19,https://api.waqi.info/feed/@10523,17.0,14.0,13.0


In [5]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pm25        1 non-null      float32       
 1   country     1 non-null      object        
 2   city        1 non-null      object        
 3   street      1 non-null      object        
 4   date        1 non-null      datetime64[ns]
 5   url         1 non-null      object        
 6   pm25_lag_1  1 non-null      float32       
 7   pm25_lag_2  1 non-null      float32       
 8   pm25_lag_3  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 188.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [6]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,-0.10,0.0,4.024922,79.695213,stockholm
1,2024-11-20,1.10,0.4,24.609364,354.122711,stockholm
2,2024-11-21,-2.70,0.2,28.401239,300.465454,stockholm
3,2024-11-22,-5.40,0.0,10.805999,271.909119,stockholm
4,2024-11-23,-3.45,0.0,13.202726,295.866302,stockholm
5,2024-11-24,2.65,0.0,15.141414,161.995743,stockholm
6,2024-11-25,5.55,0.0,15.807239,210.068497,stockholm
7,2024-11-26,7.85,0.0,22.183128,221.710663,stockholm
8,2024-11-27,1.55,0.0,9.659814,296.564972,stockholm
9,2024-11-28,2.80,0.0,16.199999,233.130020,stockholm


In [7]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [8]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-19 12:08:46,622 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169556/fs/1160259/fg/1346988


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169556/jobs/named/air_quality2_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x127f66c50>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670811
         }
       },
       "result": {
         "observed_value": 28.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T11:08:46.000621Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "k

In [10]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-19 12:09:16,597 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169556/fs/1160259/fg/1346929


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169556/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x128d81d50>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670744
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T11:09:16.000596Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
